In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")


def get_embedding(text):
    """Trích xuất vector embedding từ mô hình"""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)

    # Dùng mean pooling thay vì pooler_output
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings

def find_entity_or_relation(label, label_map, facc1_index, top_k=50, similarity_threshold=0.2):
    """
    Tìm thực thể hoặc quan hệ từ gold maps, SimCSE hoặc FACC1.
    """
    label_lower = label.lower()

    # Nếu có trong label_map, trả về ngay
    if label_map and label_lower in label_map:
        return label_map[label_lower]

    # Lấy embedding cho label
    label_embedding = get_embedding(label_lower).reshape(1, -1)

    if label_map:
        label_keys = list(label_map.keys())
        label_embeddings = np.array([get_embedding(k) for k in label_keys]).squeeze(1)

        # Đảm bảo đúng shape
        if len(label_embeddings.shape) == 1:
            label_embeddings = label_embeddings.reshape(1, -1)

        # Tính cosine similarity
        similarities = cosine_similarity(label_embedding, label_embeddings).flatten()

        # Chọn thực thể gần nhất
        merged_list = list(zip(label_keys, similarities))
        sorted_list = sorted(merged_list, key=lambda x: x[1], reverse=True)

        if sorted_list and sorted_list[0][1] > similarity_threshold:
            return label_map[sorted_list[0][0]]

    # Nếu không tìm thấy, thử trong KB (FACC1)
    facc1_cand_entities = facc1_index.get_indexrange_entity_el_pro_one_mention(label_lower, top_k=top_k)
    if facc1_cand_entities:
        best_match = max(facc1_cand_entities.items(), key=lambda x: x[1])
        return best_match[0]

    return label  # Trả về label nếu không tìm thấy

def parse_nsexpr(expr):
    """
    Chuyển chuỗi biểu thức thành cây cấu trúc dạng nested list.
    Hàm này dùng duyệt ký tự, khi gặp '(' sẽ tìm phần con cho đến khi khớp với ')',
    và giữ nguyên nội dung trong ngoặc vuông.
    """
    tokens = []
    i = 0
    while i < len(expr):
        if expr[i].isspace():
            i += 1
        elif expr[i] == '(':
            # Tìm phần con của biểu thức trong ngoặc đơn
            count = 1
            j = i + 1
            while j < len(expr) and count > 0:
                if expr[j] == '(':
                    count += 1
                elif expr[j] == ')':
                    count -= 1
                j += 1
            # Đệ quy phân tích phần con (loại bỏ ngoặc bao ngoài)
            subtree = parse_nsexpr(expr[i+1:j-1])
            tokens.append(subtree)
            i = j
        elif expr[i] == '[':
            # Giữ nguyên nội dung trong ngoặc vuông
            j = expr.find(']', i)
            if j == -1:
                return ""
                raise ValueError("Không tìm thấy dấu ']' kết thúc.")
                
            token = expr[i:j+1].strip()
            tokens.append(token)
            i = j + 1
        else:
            # Đọc một token cho đến khi gặp khoảng trắng hoặc ngoặc
            j = i
            while j < len(expr) and (not expr[j].isspace()) and expr[j] not in ['(', ')']:
                j += 1
            tokens.append(expr[i:j])
            i = j
    return tokens

def collect_labels(tree):
    """
    Duyệt cây cấu trúc (nested list) để thu thập các nhãn của quan hệ và thực thể.
    Giả sử:
      - Biểu thức JOIN có dạng: ["JOIN", relation_part, entity_part]
      - Phần relation_part: nếu là list và bắt đầu bằng "R", thì phần thứ hai chứa nhãn quan hệ (dạng "[ label ]"). 
        Nếu là chuỗi dạng "[ label ]" thì đó cũng là nhãn quan hệ.
      - Phần entity_part: nếu là chuỗi dạng "[ label ]" thì đó là nhãn thực thể, nếu là list thì xử lý đệ quy.
      - Biểu thức AND sẽ có nhiều biểu thức con.
    """
    relations = []
    entities = []
    
    if isinstance(tree, list) and tree:
        # Nếu token đầu tiên là JOIN hoặc AND
        op = tree[0]
        if isinstance(op, str):
            op_upper = op.upper()
        else:
            op_upper = ""
        
        if op_upper == "JOIN":
            # Xử lý phần quan hệ
            if len(tree) >= 2:
                rel_part = tree[1]
                # Nếu là list dạng [ "R", "[ label ]" ]
                if isinstance(rel_part, list) and len(rel_part) >= 2 and isinstance(rel_part[0], str) and rel_part[0].upper() == "R":
                    token = rel_part[1]
                    if isinstance(token, str) and token.startswith('[') and token.endswith(']'):
                        rel_label = token[1:-1].strip()
                        relations.append(rel_label)
                # Nếu là chuỗi dạng "[ label ]"
                elif isinstance(rel_part, str) and rel_part.startswith('[') and rel_part.endswith(']'):
                    rel_label = rel_part[1:-1].strip()
                    relations.append(rel_label)
                else:
                    # Nếu không đúng định dạng, duyệt đệ quy
                    sub_rel, sub_ent = collect_labels(rel_part)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
            # Xử lý phần thực thể
            if len(tree) >= 3:
                ent_part = tree[2]
                if isinstance(ent_part, list):
                    sub_rel, sub_ent = collect_labels(ent_part)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
                elif isinstance(ent_part, str) and ent_part.startswith('[') and ent_part.endswith(']'):
                    ent_label = ent_part[1:-1].strip()
                    entities.append(ent_label)
        elif op_upper == "AND":
            # Với AND, duyệt tất cả các phần con
            for sub in tree[1:]:
                sub_rel, sub_ent = collect_labels(sub)
                relations.extend(sub_rel)
                entities.extend(sub_ent)
        else:
            # Nếu không phải JOIN hay AND, duyệt tất cả các phần tử nếu chúng là list
            for elem in tree:
                if isinstance(elem, list):
                    sub_rel, sub_ent = collect_labels(elem)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
    return relations, entities



def extract_entities_and_relations(normed_expr):

    if not normed_expr or len(normed_expr) == 0:  # Kiểm tra nếu normed_expr rỗng
        return [], []
    
    if normed_expr[0] != "(":
        return [], []
    
    tree = parse_nsexpr(normed_expr)
    if tree is None:
        return [], []  # Trả về danh sách rỗng nếu parse thất bại
    
    return collect_labels(tree)

c:\Users\TOPU\anaconda3\envs\py38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\TOPU\anaconda3\envs\py38\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\TOPU\.cache\huggingface\hub\models--VoVanPhuc--sup-SimCSE-VietNamese-phobert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In orde

In [ ]:
import requests

def find_wikidata_entity(label: str, language: str = "vi"):
    """
    Tìm mã thực thể Wikidata từ nhãn, ưu tiên kết quả có nhãn khớp chính xác.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": label,
        "language": language,
        "format": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if "search" in data and data["search"]:
        results = [(item["id"], item["label"], item.get("description", "")) for item in data["search"]]
        
        # Ưu tiên kết quả có nhãn khớp chính xác trước
        # results.sort(key=lambda x: (x[1].lower() != label.lower(), len(x[2]) if x[2] else 0), reverse=True)
        
        return [item[0] for item in results]
    return None

def find_wikidata_relation(label: str, language: str = "vi"):
    """
    Tìm mã quan hệ Wikidata từ nhãn, ưu tiên kết quả có nhãn khớp chính xác.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": label,
        "language": language,
        "type": "property",  # Chỉ tìm quan hệ (property)
        "format": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if "search" in data and data["search"]:
        results = [(item["id"], item["label"], item.get("description", "")) for item in data["search"]]
        
        # # Ưu tiên kết quả có nhãn khớp chính xác trước
        # results.sort(key=lambda x: (x[1].lower() != label.lower(), len(x[2]) if x[2] else 0), reverse=True)
        
        return [item[0] for item in results]
    return None

# Ví dụ sử dụng
entity = find_wikidata_entity("South Park")

relation = find_wikidata_relation("nghề nghiệp")

print("Entity:", entity)
print("Relation:", relation)

Entity: ['Q16538', 'Q54622175', 'Q951038', 'Q1955703', 'Q2636173', 'Q4540147', 'Q650733']
Relation: ['P106']


In [3]:
import json

In [4]:
# Đọc file JSON gốc
input_file = "Data\LC-QuAD2.0\label_map\LC-QuAD2.0_test.json"
output_file = "extracted_entities_relations.json"

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)
data = data[:50]
# Xử lý từng câu nor_s_expr
results = []
for item in data:
    nor_s_expr = item.get("nor_s_expr", "")
    gold_rel = item.get("gold_relation_map")
    gold_ent = item.get("gold_entity_map")
    rel, ent = extract_entities_and_relations(nor_s_expr)
    r_list =[]
    e_list = []
    for r in rel:
        r_list.append(find_wikidata_relation(r))
    for e in ent:
        e_list.append(find_wikidata_entity(e))
    results.append({"input": nor_s_expr, "relation":r_list,"entity": e_list, "gold_rel":gold_rel,"gold_ent": gold_ent})

# Lưu kết quả ra file mới
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print(f"Đã lưu kết quả vào {output_file}")

Đã lưu kết quả vào extracted_entities_relations.json
